# VacationPy 
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Suchiapa,16.6167,-93.0833,80.92,39,0,39.12,MX,1707170689
1,1,Adamstown,-25.0660,-130.1015,75.63,82,100,15.35,PN,1707170690
2,2,Lakhipur,26.0299,90.3041,56.50,79,42,7.29,IN,1707170690
3,3,Papatowai,-46.5619,169.4708,70.48,56,100,9.55,NZ,1707170690
4,4,Qaqortoq,60.7167,-46.0333,-6.03,69,0,6.26,GL,1707170690


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 33 degrees but higher than 22
# Humidity less than or equal to 50
# Limited clouds in the sky ( cloudiness < 25)

ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <= 33) & (city_data_df["Max Temp"] >= 22)&
                               (city_data_df["Humidity"] >= 50) & (city_data_df["Cloudiness"] < 25)]

# Drop any rows with null values
ideal_weather_df=ideal_weather.dropna(axis=0 , how="any")

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,Sandnessjoen,66.0217,12.6316,24.91,93,20,2.30,NO,1707170760
351,351,Quchan,37.1060,58.5095,31.93,74,9,6.49,IR,1707170788
545,545,Bodo,67.2800,14.4050,24.78,74,0,16.11,NO,1707170807
698,698,Turpan,42.9333,89.1667,31.06,50,5,2.24,CN,1707170821
724,724,Dowlatabad,32.7998,51.6955,26.28,74,0,4.61,IR,1707170823


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df[["City", "Country","Lat","Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/var/folders/48/p0hf_jfn6vg9k6p3wqpjj7qc0000gn/T/ipykernel_33338/960165084.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
102,Sandnessjoen,NO,66.0217,12.6316,93,
351,Quchan,IR,37.1060,58.5095,74,
545,Bodo,NO,67.2800,14.4050,74,
698,Turpan,CN,42.9333,89.1667,50,
724,Dowlatabad,IR,32.7998,51.6955,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel',
          "limit": 1,
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Sandnessjoen - nearest hotel: Scandic Syv Søstre
Quchan - nearest hotel: هتل خیام
Bodo - nearest hotel: Zefyr hotell
Turpan - nearest hotel: Oriental
Dowlatabad - nearest hotel: هتل آفتاب اصفهان


,City,Country,Lat,Lng,Humidity,Hotel Name
102,Sandnessjoen,NO,66.0217,12.6316,93,Scandic Syv Søstre
351,Quchan,IR,37.1060,58.5095,74,هتل خیام
545,Bodo,NO,67.2800,14.4050,74,Zefyr hotell
698,Turpan,CN,42.9333,89.1667,50,Oriental
724,Dowlatabad,IR,32.7998,51.6955,74,هتل آفتاب اصفهان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=0.5,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)